## 0. 준비 

In [1]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#라이브러리 import
import pandas as pd
import numpy as np
import os
import moviepy.editor
import requests
import json
from moviepy.editor import VideoFileClip, concatenate_videoclips

경로 설정

In [3]:
#폴더 생성
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [4]:
movie_name="The Girl Who Believes in Miracles" # 영화 제목 입력
mydrive="/content/drive/MyDrive/gp/"
createFolder(mydrive+movie_name) # 영화 제목 폴더 생성

In [5]:
#비디오 클립 경로 지정
createFolder(mydrive+movie_name+"/clip") 
video_path= mydrive+movie_name+"/"+movie_name+".mp4"

## 1. STT API

In [ ]:
#비디오를 오디오로 바꾸는 과정
clip = VideoFileClip(video_path)
audio=clip.audio.write_audiofile(mydrive+movie_name+"/audio.mp3")

In [ ]:
class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = '      '
    # Clova Speech secret key
    secret = '          '

    def req_url(self, url, completion, callback=None, userdata=None, forbiddens=None, boostings=None, wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'url': url,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/url',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_object_storage(self, data_key, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                           wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'dataKey': data_key,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/object-storage',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

if __name__ == '__main__':
    res = ClovaSpeechClient().req_upload(file=audio, completion='sync')
    print(res.text)


##2. 데이터 전처리

In [6]:
#데이터 로드
file_path= mydrive+movie_name+"/"+movie_name
df = pd.read_csv(file_path+'_stt.csv')
df.head()

,Index,Modification,Speaker,Start Time,End Time,Script
0,1,인식결과,NaN,4160,17940,NaN
1,1,수정결과,NaN,4160,17940,NaN
2,2,인식결과,NaN,19600,20780,NaN
3,2,수정결과,NaN,19600,20780,NaN
4,3,인식결과,NaN,22860,23960,NaN


1. 인식결과 행만 남김 (수행결과 행 삭제)

In [7]:
df=df.assign(g=df.index % 2).query('g==0') # 짝수행만 남김
df.head() 

,Index,Modification,Speaker,Start Time,End Time,Script,g
0,1,인식결과,NaN,4160,17940,NaN,0
2,2,인식결과,NaN,19600,20780,NaN,0
4,3,인식결과,NaN,22860,23960,NaN,0
6,4,인식결과,NaN,25140,26260,NaN,0
8,5,인식결과,NaN,29300,31720,NaN,0


2. 필요 없는 colums 삭제

In [8]:
df=df.drop(columns='g')  #g행 삭제
df=df.drop('Speaker',axis='columns') # speaker 열 삭제
df= df.drop('Modification',axis='columns') #modification 열 삭제
df= df.drop('Index',axis='columns') #modification 열 삭제

3. Script에 결측치 있는 colums삭제

In [9]:
df=df.dropna(axis='index',how='any')

4.  인덱스 초기화

In [10]:
df= df.reset_index(drop=True)

In [11]:
df.head() 
df.to_csv('STT.csv', index=False, encoding='cp949')

## 3.New Time Stemp 

In [12]:
df['start'] = df.apply(lambda row : (row['Start Time']+ row['End Time'])/2-5000, axis = 1)
df['end'] = df.apply(lambda row : (row['Start Time']+ row['End Time'])/2+5000, axis = 1)

df=df.drop('Start Time',axis='columns')
df=df.drop('End Time',axis='columns')

df.head() 

,Script,start,end
0,It is perfectly typical for most people to hav...,29970.0,39970.0
1,relying on them for love and support. These ar...,37770.0,47770.0
2,Let's look at another parable that talks about...,45060.0,55060.0
3,And here's what it says.,47880.0,57880.0
4,The kingdom of heaven is like a grain of a mus...,53870.0,63870.0


## 4.text(대사) list 제작 

In [13]:
df_temp=df #임의 배열 만듬
df_temp=df_temp.drop('start',axis='columns') 
df_temp=df_temp.drop('end',axis='columns')

In [14]:
df_temp.to_csv(file_path+'_text_list.csv',index=True) #csv파일로 저장

## 5.비디오 clip 제작

In [15]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
#clip 저장 코드
n=0
for start, end in zip(df['start'], df['end']):
  clip = VideoFileClip(video_path).subclip(start*0.001,end*0.001)
  clip.write_videofile(mydrive+movie_name+"/clip/"+str(n)+".mp4")
  print(str(n)+"번째 클립 저장 완료")
  n=n+1

[MoviePy] >>>> Building video /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/0.mp4
[MoviePy] Writing audio in 0TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:01<00:00, 220.51it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/0.mp4



100%|██████████| 240/240 [00:03<00:00, 70.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/0.mp4 

0번째 클립 저장 완료
[MoviePy] >>>> Building video /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/1.mp4
[MoviePy] Writing audio in 1TEMP_MPY_wvf_snd.mp3


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/678.mp4 

678번째 클립 저장 완료
[MoviePy] >>>> Building video /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/679.mp4
[MoviePy] Writing audio in 679TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:00<00:00, 423.79it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/679.mp4



100%|██████████| 240/240 [00:03<00:00, 65.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/gp/The Girl Who Believes in Miracles/clip/679.mp4 

679번째 클립 저장 완료
